In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["WEATHER_API_KEY"] = os.getenv("WEATHER_API_KEY")
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

In [2]:
import requests

def get_weather_from_weatherapi(city: str) -> str:
    """A funtion to track weather"""
    try:
        url = f"http://api.weatherapi.com/v1/forecast.json?key={WEATHER_API_KEY}&q={city}&days=3&aqi=no&alerts=no"
        response = requests.get(url)
        
        # Ensure valid JSON
        if response.status_code != 200:
            return f"❗ API request failed with status code {response.status_code}"
        
        data = response.json()

        if "error" in data:
            return f"❗ API Error: {data['error'].get('message', 'Unknown error')}"

        location = data["location"]
        current = data["current"]
        forecast = data["forecast"]["forecastday"]

        report = (
            f"📍 {location['name']}, {location['country']}\n"
            f"🌡️ Temp: {current['temp_c']}°C (Feels like {current['feelslike_c']}°C)\n"
            f"💧 Humidity: {current['humidity']}% | ☁️ Cloud: {current['cloud']}%\n"
            f"🌬️ Wind: {current['wind_kph']} kph\n"
            f"🌤️ Condition: {current['condition']['text']}\n\n"
            f"🗓️ Forecast:\n"
        )

        for day in forecast:
            date = day["date"]
            day_data = day["day"]
            report += (
                f"{date}: 🌡️ {day_data['avgtemp_c']}°C, 🌧️ {day_data['condition']['text']}, "
                f"💧 Humidity: {day_data['avghumidity']}%, 🌬️ Wind: {day_data['maxwind_kph']} kph\n"
            )

        return report

    except Exception as e:
        return f"❗ Error: {str(e)}"

In [3]:
from langgraph.graph import StateGraph, END, START
from langchain_groq import ChatGroq

llm = ChatGroq(model="deepseek-r1-distill-llama-70b")
llm_with_tools = llm.bind_tools([get_weather_from_weatherapi])

In [4]:
from typing_extensions import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages : Annotated[list[AnyMessage], add_messages]

In [ ]:
from langgraph.prebuilt import create_react_agent

weather_agent = create_react_agent(
        model= llm_with_tools,
        tools= [get_weather_from_weatherapi],
        prom
    )

In [8]:
response = weather_agent.invoke({"messages":"Banglore"})

In [17]:
for event in response["messages"]:
    print(event.content)

Banglore

📍 Bangalore, India
🌡️ Temp: 26.3°C (Feels like 27.5°C)
💧 Humidity: 74% | ☁️ Cloud: 75%
🌬️ Wind: 29.5 kph
🌤️ Condition: Light rain

🗓️ Forecast:
2025-06-17: 🌡️ 22.9°C, 🌧️ Patchy rain nearby, 💧 Humidity: 77%, 🌬️ Wind: 32.0 kph
2025-06-18: 🌡️ 23.3°C, 🌧️ Partly Cloudy , 💧 Humidity: 72%, 🌬️ Wind: 29.5 kph
2025-06-19: 🌡️ 23.5°C, 🌧️ Partly Cloudy , 💧 Humidity: 71%, 🌬️ Wind: 28.8 kph


📍 Bangalore, India
🌡️ Temp: 26.3°C (Feels like 27.5°C)
💧 Humidity: 74% | ☁️ Cloud: 75%
🌬️ Wind: 29.5 kph
🌤️ Condition: Light rain

🗓️ Forecast:
2025-06-17: 🌡️ 22.9°C, 🌧️ Patchy rain nearby, 💧 Humidity: 77%, 🌬️ Wind: 32.0 kph
2025-06-18: 🌡️ 23.3°C, 🌧️ Partly Cloudy , 💧 Humidity: 72%, 🌬️ Wind: 29.5 kph
2025-06-19: 🌡️ 23.5°C, 🌧️ Partly Cloudy , 💧 Humidity: 71%, 🌬️ Wind: 28.8 kph


